# Previsão de Preço com LSTM - Ação VALE3

In [ ]:
# Instalar bibliotecas necessárias (se ainda não instaladas)
!pip install yfinance ta scikit-learn tensorflow

In [ ]:
import yfinance as yf
import ta
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Baixar dados da VALE3 com intervalo de 1 minuto
symbol = 'VALE3.SA'
df = yf.download(tickers=symbol, period='7d', interval='1m', progress=True)

In [ ]:
# Criar indicadores técnicos
close = df['Close'].squeeze()
df['SMA_10'] = ta.trend.sma_indicator(close, window=10)
df['SMA_20'] = ta.trend.sma_indicator(close, window=20)
df['RSI_14'] = ta.momentum.rsi(close, window=14)
df['MACD'] = ta.trend.macd(close)

In [ ]:
# Tratar valores faltantes
df.dropna(inplace=True)

In [ ]:
# Selecionar colunas
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_10', 'SMA_20', 'RSI_14', 'MACD']
data = df[features]

In [ ]:
# Escalar os dados
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:
# Criar sequências para o LSTM
def create_sequences(data, seq_length=60):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i, features.index('Close')])
    return np.array(X), np.array(y)

SEQ_LENGTH = 60
X, y = create_sequences(data_scaled, SEQ_LENGTH)

In [ ]:
# Dividir treino/teste
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# Criar modelo LSTM
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Treinar modelo
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[early_stop])

In [ ]:
# Fazer previsões
y_pred_scaled = model.predict(X_test)

In [ ]:
# Desescalar as previsões
close_index = features.index('Close')
close_min = scaler.min_[close_index]
close_scale = scaler.scale_[close_index]

y_pred = y_pred_scaled * close_scale + close_min
y_true = y_test * close_scale + close_min

In [ ]:
# Avaliar desempenho
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = mean_absolute_percentage_error(y_true, y_pred)

print(f"MAE: {mae:.5f}")
print(f"RMSE: {rmse:.5f}")
print(f"MAPE: {mape:.5f}")